## Homework: Vector Search

In this homework, we will learn more about vector search and embedding. Like in the module, we will use Qdrant and fastembed

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

Note: if you want to learn how vector search works under the hood, check [homework 3 from 2024 cohort (questions 1-4)](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/03-vector-search/homework.md)

## Embeddings

Qdrant uses [fastembed](https://github.com/qdrant/fastembed) under the hood to turn text into vectors.

We will now explore this library

Make sure it's installed:
```
pip install fastembed
```
Import it:
```
from fastembed import TextEmbedding
```
### Q1. Embedding the query

Embed the query: ```'I just discovered the course. Can I join now?'```. Use the ```'jinaai/jina-embeddings-v2-small-en'``` model.

You should get a numpy array of size 512.

What's the minimal value in this array?

* -0.51
* **-0.11**
* 0
* 0.51


In [1]:
from fastembed import TextEmbedding

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

raw_query = "I just discovered the course. Can I join now?"
prefixed_query = f"query: {raw_query}"

Fetching 5 files: 100%|████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.59it/s]


In [6]:
# Embed your query
# embed() returns a generator of numpy arrays
emb_gen = model.embed([prefixed_query])
query_embedding = next(emb_gen)  # or: list(model.embed([prefixed_query]))[0]

In [7]:
# Use the embedded
print(f"Embedding vector (shape {query_embedding.shape}):\n", query_embedding)

Embedding vector (shape (512,)):
 [-5.56700644e-02 -5.91947790e-02  5.39899518e-02  3.80470234e-02
 -4.92856502e-03 -3.47944135e-02  3.99072577e-02  4.59148763e-02
  1.83341418e-02 -9.52772089e-03  4.54004942e-02 -1.20338823e-02
 -4.21386614e-02 -1.08453106e-01  7.23750023e-02 -3.05278842e-02
 -9.65327805e-03 -2.90670041e-02 -3.98645727e-02 -1.72941270e-02
 -2.31294233e-02 -6.67832059e-03  2.59414772e-02 -3.59517180e-03
  8.23218558e-02 -8.10587483e-02 -6.34000685e-02  5.10138388e-02
  4.07947449e-02  6.18107363e-02 -3.70725083e-02  2.81571896e-02
  2.46928692e-03  1.11272689e-02 -4.52913732e-02  2.17225107e-02
  5.10061227e-02  1.35064928e-02 -2.41472429e-02 -4.36897430e-02
  6.06906639e-03  2.94652879e-02  7.32580567e-02  2.75495004e-02
 -5.47573179e-02 -7.90277460e-03 -1.16249444e-01  3.71479592e-02
  1.71589254e-02 -1.08521203e-02 -5.73682334e-02  3.52678604e-02
 -8.04892372e-02  4.22803926e-02  1.26776605e-02  4.97862691e-02
 -3.61782539e-02  3.30159532e-02 -4.80931209e-02 -6.0442

In [8]:
query_embedding.min()

np.float64(-0.11624944375698411)

## Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the norm function:
```python
import numpy as np
np.linalg.norm(q)
```
Which means that we can simply compute the dot product between two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:
```python
q.dot(q)
```
## Q2. Cosine similarity with another vector

Now let's embed this document:

```doc = 'Can I still join the course after the start date?'```

What's the cosine similarity between the vector for the query and the vector for the document?

* 0.3
* 0.5
* 0.7
* **0.9**


In [10]:
import numpy as np

raw_doc_query = "Can I still join the course after the start date?"
prefixed_query = f"query: {raw_doc_query}"

emb_doc_gen = model.embed([prefixed_query])
doc_embedding = next(emb_doc_gen)  

In [14]:
query_embedding.dot(doc_embedding) / np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding)

np.float64(0.9013799853601606)

## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:
```
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]
```
Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

* 0
* 1
* **2**
* 3
* 4

Hint: if you put all the embeddings of the text field in one matrix V (a single 2-dimensional numpy array), then computing the cosine becomes a matrix multiplication:
```python
V.dot(q)
```
If this hint is rather confusing you than helping, feel free to ignore it.

In [15]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [16]:
# Prepare and embed passages
passages = [f"passage: {doc['text']}" for doc in documents]
passage_embeddings = list(model.embed(passages))

# Compute cosine similarities against your query embedding
def cosine(u, v):
    return u.dot(v) / (np.linalg.norm(u) * np.linalg.norm(v))

sims = [cosine(query_embedding, pe) for pe in passage_embeddings]

# Find the index of the most similar document
best_idx = int(np.argmax(sims))
print("Most similar document index:", best_idx)

Most similar document index: 2


## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of question and text:
```
full_text = doc['question'] + ' ' + doc['text']
```
Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

* **0** *In short, concatenating the question changes the embedding, so it’s completely normal that the top-hit index shifts when you switch from “text only” to “question + text.”*
* 1
* 2
* 3
* 4

Is it different from Q3? If yes, why?

In [20]:
concat_passages = [
    f"passage: {doc['question']}  {doc['text']}"
    for doc in documents
]
concat_passage_embeddings = list(model.embed(concat_passages))

concat_sims = [cosine(query_embedding, cpe) for cpe in concat_passage_embeddings]

concat_best_idx = int(np.argmax(concat_sims))
print("Most similar document index:", concat_best_idx)

Most similar document index: 0


## Q5. Selecting the embedding model

Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

* 128
* 256
* **384**
* 512

One of these models is ```BAAI/bge-small-en```. Let's use it.

In [23]:
import pandas as pd

df_text  = pd.DataFrame(TextEmbedding.list_supported_models())
min_text_dim  = df_text['dim'].min()

print("Smallest TextEmbedding dim:         ", min_text_dim)

print("\nTextEmbedding model(s) with dim =", min_text_dim, ":")
print(df_text[df_text['dim']==min_text_dim]['model'].tolist())

Smallest TextEmbedding dim:          384

TextEmbedding model(s) with dim = 384 :
['BAAI/bge-small-en', 'BAAI/bge-small-en-v1.5', 'snowflake/snowflake-arctic-embed-xs', 'snowflake/snowflake-arctic-embed-s', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2']


## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:
```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```
Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

```text = doc['question'] + ' ' + doc['text']```

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results? (The score for the first returned record):

* 0.97
* 0.87
* 0.77
* 0.67

In [24]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
